In [1]:
pip install openpyxl

Note: you may need to restart the kernel to use updated packages.


# Import Libraries we will use

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import openpyxl

# Knowing the name of our Excel sheet

In [3]:
def get_excel_sheet_names(file_path):
    # Load the Excel file
    xls = pd.ExcelFile(file_path)
    
    # Retrieve the sheet names
    sheet_names = xls.sheet_names
    
    return sheet_names

# Example usage
file_path = 'KPMG_VI_New_raw_data_update_final.xlsx'
sheet_names = get_excel_sheet_names(file_path)

print(sheet_names)

['Title Sheet', 'Transactions', 'NewCustomerList', 'CustomerDemographic', 'CustomerAddress']


# Upload our Dataset

In [4]:
transactions = pd.read_excel(file_path, sheet_name='Transactions')
newcustomerlist = pd.read_excel(file_path, sheet_name='NewCustomerList')
customerdemographic = pd.read_excel(file_path, sheet_name='CustomerDemographic')
customeraddress = pd.read_excel(file_path, sheet_name='CustomerAddress')
print('upload successfully')

upload successfully


# Understand and cleaning out Dataset

## First : Transaction Dataset 

In [5]:
transactions.head()

,transaction_id,product_id,customer_id,transaction_date,online_order,order_status,brand,product_line,product_class,product_size,list_price,standard_cost,product_first_sold_date
0,1,2,2950,2017-02-25,0.0,Approved,Solex,Standard,medium,medium,71.49,53.62,41245.0
1,2,3,3120,2017-05-21,1.0,Approved,Trek Bicycles,Standard,medium,large,2091.47,388.92,41701.0
2,3,37,402,2017-10-16,0.0,Approved,OHM Cycles,Standard,low,medium,1793.43,248.82,36361.0
3,4,88,3135,2017-08-31,0.0,Approved,Norco Bicycles,Standard,medium,medium,1198.46,381.10,36145.0
4,5,78,787,2017-10-01,1.0,Approved,Giant Bicycles,Standard,medium,large,1765.30,709.48,42226.0


In [6]:
transactions.shape

(20000, 13)

### Sure that the data in the correct Type : 

In [7]:
transactions.dtypes

transaction_id                      int64
product_id                          int64
customer_id                         int64
transaction_date           datetime64[ns]
online_order                      float64
order_status                       object
brand                              object
product_line                       object
product_class                      object
product_size                       object
list_price                        float64
standard_cost                     float64
product_first_sold_date           float64
dtype: object

### Dealing with missing Data :

In [8]:
transactions.isna().sum()

transaction_id               0
product_id                   0
customer_id                  0
transaction_date             0
online_order               360
order_status                 0
brand                      197
product_line               197
product_class              197
product_size               197
list_price                   0
standard_cost              197
product_first_sold_date    197
dtype: int64

In [9]:
threhold = len(transactions) * 0.05
print("threhold : ",threhold)
cols_to_drop = transactions.columns[transactions.isna().sum() <= threhold ]
transactions.dropna(subset=cols_to_drop,inplace=True)
transactions.isna().sum()

threhold :  1000.0


transaction_id             0
product_id                 0
customer_id                0
transaction_date           0
online_order               0
order_status               0
brand                      0
product_line               0
product_class              0
product_size               0
list_price                 0
standard_cost              0
product_first_sold_date    0
dtype: int64

## Cleaning the Transaction Dataset

### We insure that the Data contains allowable Value :

In [10]:
transactions.columns

Index(['transaction_id', 'product_id', 'customer_id', 'transaction_date',
       'online_order', 'order_status', 'brand', 'product_line',
       'product_class', 'product_size', 'list_price', 'standard_cost',
       'product_first_sold_date'],
      dtype='object')

In [11]:
transactions['transaction_date'].dt.year.unique()

array([2017])

In [12]:
transactions['order_status'].unique()

array(['Approved', 'Cancelled'], dtype=object)

In [13]:
transactions['brand'].unique()

array(['Solex', 'Trek Bicycles', 'OHM Cycles', 'Norco Bicycles',
       'Giant Bicycles', 'WeareA2B'], dtype=object)

In [14]:
transactions['product_line'].unique()

array(['Standard', 'Road', 'Mountain', 'Touring'], dtype=object)

In [15]:
transactions['product_class'].unique()

array(['medium', 'low', 'high'], dtype=object)

In [16]:
transactions['product_size'].unique()

array(['medium', 'large', 'small'], dtype=object)

### Last step insure that is no dublication in our data:

In [17]:
transactions.duplicated().sum()

0

### We now sure that Transaction Dataset is Cleaned and ready to next step
## second : The newcustomerlist Dataset

In [25]:
newcustomerlist.head()

,first_name,last_name,gender,past_3_years_bike_related_purchases,DOB,job_title,job_industry_category,wealth_segment,deceased_indicator,owns_car,...,state,country,property_valuation,Unnamed: 16,Unnamed: 17,Unnamed: 18,Unnamed: 19,Unnamed: 20,Rank,Value
0,Chickie,Brister,Male,86,1957-07-12,General Manager,Manufacturing,Mass Customer,N,Yes,...,QLD,Australia,6,0.80,1.000,1.2500,1.0625,1,1,1.718750
1,Morly,Genery,Male,69,1970-03-22,Structural Engineer,Property,Mass Customer,N,No,...,NSW,Australia,11,0.60,0.600,0.7500,0.6375,1,1,1.718750
2,Ardelis,Forrester,Female,10,1974-08-28 00:00:00,Senior Cost Accountant,Financial Services,Affluent Customer,N,No,...,VIC,Australia,5,0.99,0.990,0.9900,0.9900,1,1,1.718750
3,Lucine,Stutt,Female,64,1979-01-28,Account Representative III,Manufacturing,Affluent Customer,N,Yes,...,QLD,Australia,1,1.02,1.275,1.2750,1.2750,4,4,1.703125
4,Melinda,Hadlee,Female,34,1965-09-21,Financial Analyst,Financial Services,Affluent Customer,N,No,...,NSW,Australia,9,1.01,1.010,1.2625,1.2625,4,4,1.703125


In [24]:
newcustomerlist.shape

(1000, 23)

In [27]:
newcustomerlist.columns

Index(['first_name', 'last_name', 'gender',
       'past_3_years_bike_related_purchases', 'DOB', 'job_title',
       'job_industry_category', 'wealth_segment', 'deceased_indicator',
       'owns_car', 'tenure', 'address', 'postcode', 'state', 'country',
       'property_valuation', 'Unnamed: 16', 'Unnamed: 17', 'Unnamed: 18',
       'Unnamed: 19', 'Unnamed: 20', 'Rank', 'Value'],
      dtype='object')

### We found columns that is hidden in excel we will not use we remove it

In [29]:
newcustomerlist = newcustomerlist.drop(['Unnamed: 16', 'Unnamed: 17', 'Unnamed: 18',
       'Unnamed: 19', 'Unnamed: 20'],axis=1)
newcustomerlist.columns

Index(['first_name', 'last_name', 'gender',
       'past_3_years_bike_related_purchases', 'DOB', 'job_title',
       'job_industry_category', 'wealth_segment', 'deceased_indicator',
       'owns_car', 'tenure', 'address', 'postcode', 'state', 'country',
       'property_valuation', 'Rank', 'Value'],
      dtype='object')

In [30]:
newcustomerlist.dtypes

first_name                              object
last_name                               object
gender                                  object
past_3_years_bike_related_purchases      int64
DOB                                     object
job_title                               object
job_industry_category                   object
wealth_segment                          object
deceased_indicator                      object
owns_car                                object
tenure                                   int64
address                                 object
postcode                                 int64
state                                   object
country                                 object
property_valuation                       int64
Rank                                     int64
Value                                  float64
dtype: object

### DOB should be a date not an object

In [31]:
newcustomerlist['DOB'] = pd.to_datetime(newcustomerlist['DOB'])

In [35]:
newcustomerlist.dtypes

first_name                                     object
last_name                                      object
gender                                         object
past_3_years_bike_related_purchases             int64
DOB                                    datetime64[ns]
job_title                                      object
job_industry_category                          object
wealth_segment                                 object
deceased_indicator                             object
owns_car                                       object
tenure                                          int64
address                                        object
postcode                                        int64
state                                          object
country                                        object
property_valuation                              int64
Rank                                            int64
Value                                         float64
dtype: object

### Deal with missing value

In [36]:
newcustomerlist.isna().sum()

first_name                               0
last_name                               29
gender                                   0
past_3_years_bike_related_purchases      0
DOB                                     17
job_title                              106
job_industry_category                  165
wealth_segment                           0
deceased_indicator                       0
owns_car                                 0
tenure                                   0
address                                  0
postcode                                 0
state                                    0
country                                  0
property_valuation                       0
Rank                                     0
Value                                    0
dtype: int64

## Remove rows contains missing data but if it's less than 5% of the Dataset

In [37]:
threhold = len(newcustomerlist) * 0.05
print("threhold : ",threhold)
cols_to_drop = newcustomerlist.columns[newcustomerlist.isna().sum() <= threhold ]
newcustomerlist.dropna(subset=cols_to_drop,inplace=True)
newcustomerlist.isna().sum()

threhold :  50.0


first_name                               0
last_name                                0
gender                                   0
past_3_years_bike_related_purchases      0
DOB                                      0
job_title                              102
job_industry_category                  159
wealth_segment                           0
deceased_indicator                       0
owns_car                                 0
tenure                                   0
address                                  0
postcode                                 0
state                                    0
country                                  0
property_valuation                       0
Rank                                     0
Value                                    0
dtype: int64

### We have a 2 Categorical columns we replace the Null value with the mode 

In [48]:
mode1 = newcustomerlist['job_title'].mode()
mode2 = newcustomerlist['job_industry_category'].mode()
print(mode1)
print(' ')
print(mode2)

0    Associate Professor
1     Environmental Tech
2    Software Consultant
Name: job_title, dtype: object
 
0    Manufacturing
Name: job_industry_category, dtype: object


In [52]:
mode1 = mode1.to_numpy()
mode2 =mode2.to_numpy()
print('put them in an array successfully')

put them in an array successfully


### We have 3 modes in one column we choose to fill the missing value randomly between them

In [53]:
if newcustomerlist['job_title'].isna().any():
    newcustomerlist['job_title']= newcustomerlist['job_title'].fillna(np.random.choice(mode1))

In [55]:
if newcustomerlist['job_industry_category'].isna().any():
    newcustomerlist['job_industry_category']= newcustomerlist['job_industry_category'].fillna(np.random.choice(mode2))

### Finaly we clean our Missing values

In [56]:
newcustomerlist.isna().sum()

first_name                             0
last_name                              0
gender                                 0
past_3_years_bike_related_purchases    0
DOB                                    0
job_title                              0
job_industry_category                  0
wealth_segment                         0
deceased_indicator                     0
owns_car                               0
tenure                                 0
address                                0
postcode                               0
state                                  0
country                                0
property_valuation                     0
Rank                                   0
Value                                  0
dtype: int64

## We insure that the Data contains allowable Value :

In [57]:
newcustomerlist['gender'].unique()

array(['Male', 'Female'], dtype=object)

In [58]:
newcustomerlist['DOB'].dt.year.unique()

array([1957, 1970, 1974, 1979, 1965, 1951, 1976, 1972, 1985, 1995, 1999,
       1964, 1954, 1993, 1938, 1968, 1998, 1977, 1982, 1945, 1952, 1966,
       1960, 1980, 1987, 1981, 1975, 1955, 1953, 1946, 1961, 1956, 1994,
       1950, 1978, 1983, 1971, 1942, 1967, 1996, 1984, 1988, 1958, 1973,
       1949, 1992, 1943, 2000, 1991, 1940, 1963, 1969, 1939, 1997, 1941,
       1990, 1962, 2001, 1986, 1989, 1944, 1959, 1948, 1947, 2002])

In [59]:
newcustomerlist['wealth_segment'].unique()

array(['Mass Customer', 'Affluent Customer', 'High Net Worth'],
      dtype=object)

In [60]:
newcustomerlist['deceased_indicator'].unique()

array(['N'], dtype=object)

In [61]:
newcustomerlist['owns_car'].unique()

array(['Yes', 'No'], dtype=object)

In [64]:
newcustomerlist['state'].unique()

array(['QLD', 'NSW', 'VIC'], dtype=object)

In [65]:
newcustomerlist['country'].unique()

array(['Australia'], dtype=object)

### Last step insure that is no dublication in our data:

In [68]:
newcustomerlist.duplicated().sum()

0

### We now sure that newcustomerlist Dataset is Cleaned and ready to next step
## Third : The customerdemographic Dataset